#1.Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/Grammatical_Error_correction/

/content/drive/MyDrive/Grammatical_Error_correction


#2.Import and Installations

In [ ]:
import pandas as pd

#3.Download data

wget used to get data from web. 
syntax: wget <path> -P <path> <br>
-P --> path to download file

In [ ]:
!wget https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz -P /content/drive/MyDrive/Grammatical_Error_correction/Data/

--2021-06-21 15:02:29--  https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz
Resolving www.comp.nus.edu.sg (www.comp.nus.edu.sg)... 45.60.35.225
Connecting to www.comp.nus.edu.sg (www.comp.nus.edu.sg)|45.60.35.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 643482 (628K) [application/x-gzip]
Saving to: ‘/content/drive/MyDrive/Grammatical_Error_correction/Data/conll14st-test-data.tar.gz.1’

conll14st-test-data 100%[===================>] 628.40K   303KB/s    in 2.1s    

2021-06-21 15:02:32 (303 KB/s) - ‘/content/drive/MyDrive/Grammatical_Error_correction/Data/conll14st-test-data.tar.gz.1’ saved [643482/643482]



In [ ]:
!pwd

/content/drive/My Drive/Grammatical_Error_correction


In [ ]:
!tar -xvzf '/content/drive/My Drive/Grammatical_Error_correction/Data/conll14st-test-data.tar.gz' -C '/content/drive/My Drive/Grammatical_Error_correction/Data/'

conll14st-test-data/
conll14st-test-data/scripts/
conll14st-test-data/scripts/parser_feature.py
conll14st-test-data/scripts/preprocess.py
conll14st-test-data/scripts/nuclesgmlparser.py
conll14st-test-data/scripts/README
conll14st-test-data/scripts/nucle_doc.py
conll14st-test-data/scripts/preprocesscombine.py
conll14st-test-data/scripts/preprocesswithalt.py
conll14st-test-data/scripts/iparser.py
conll14st-test-data/noalt/
conll14st-test-data/noalt/official-2014.1.conll.ann
conll14st-test-data/noalt/official-2014.0.sgml
conll14st-test-data/noalt/official-2014.combined.m2
conll14st-test-data/noalt/official-2014.1.sgml
conll14st-test-data/noalt/official-2014.1.m2
conll14st-test-data/noalt/official-2014.0.conll.ann
conll14st-test-data/noalt/official-2014.0.m2
conll14st-test-data/README
conll14st-test-data/alt/
conll14st-test-data/alt/alternative-teama.sgml
conll14st-test-data/alt/alternative-teamc.sgml
conll14st-test-data/alt/alternative-teamb.sgml
conll14st-test-data/alt/official-2014.comb

#4.Read data and Prepare data

below code snippet is taken from: <br> https://gist.github.com/mridul1012/9867d3b5dc77e0f12d5568b47b7ff9a2#file-m2_to_csv-py
  https://www.cl.cam.ac.uk/research/nl/bea2019st/data/corr_from_m2.py

In [ ]:
def process_m2file(m2_file, out_file, id=0):

  """
  This function used to convert .m2 file to .txt file. 
  This functions return dataframe with correct, incorrect, total_edits 

  Arguments:
  m2_file: path to .m2 file
  out: path to .txt file
  id: Annotator id

  converts specified annotators text from erroneous to non-erroneous
  """
  
  m2 = open(m2_file).read().strip().split("\n\n")
  out = open(out_file, "w")
  # Do not apply edits with these error types
  skip = {"noop", "UNK", "Um"}
  total_edits = []
  
  for sent in m2:
    sent = sent.split("\n")
    cor_sent = sent[0].split()[1:] # Ignore "S "
    edits = sent[1:]
    offset = 0
    cur_edit = 0
    for edit in edits:
      edit = edit.split("|||")
      if edit[1] in skip: continue # Ignore certain edits
      coder = int(edit[-1])
      if coder != id: continue # Ignore other coders
      span = edit[0].split()[1:] # Ignore "A "
      start = int(span[0])
      end = int(span[1])
      cor = edit[2].split()
      cor_sent[start+offset:end+offset] = cor
      cur_edit+=1
      offset = offset-(end-start)+len(cor)
    total_edits.append(cur_edit)
    out.write(" ".join(cor_sent)+"\n")
  out.close()

  file1 = open(m2_file,"r")
  s1 = file1.read()

  each_sent = s1.split("\n\n")

  incorrect = []
  for i in range(len(each_sent)):
      temp = each_sent[i].split("\n")
      temp = temp[0]
      temp = temp.split(" ")
      temp = temp[1:]# ignore S
      temp = ' '.join(temp)
      incorrect.append(temp)
      
  #preprocessing correct sentences

  file2 = open(out_file, "r")
  s2 = file2.read()

  correct = s2.split("\n")

  print("Preparing dataframe")
  df = pd.DataFrame()
  df["correct"] = correct
  df["incorrect"] = incorrect
  total_edits.append(0)
  df['total_edits'] = total_edits

  return df

## Extracting data from Annatator 0

In [ ]:
data_id0 = process_m2file(m2_file='/content/drive/My Drive/Grammatical_Error_correction/Data/conll14st-test-data/noalt/official-2014.combined.m2',
                      out_file='/content/drive/My Drive/Grammatical_Error_correction/Data/conll14st-test-data/noalt/official-2014.combined.m2_id0_corrected.txt',
                      id=0)

In [ ]:
data_id0.shape

(1313, 3)

In [ ]:
#save dataframe
data_id0.to_csv('/content/drive/MyDrive/Grammatical_Error_correction/Data/csv_data/conll14_combined_id0.csv', index=False)

## Extracting data from Annotator 1

In [ ]:
data_id1 = process_m2file(m2_file='/content/drive/My Drive/Grammatical_Error_correction/Data/conll14st-test-data/noalt/official-2014.combined.m2',
                      out_file='/content/drive/My Drive/Grammatical_Error_correction/Data/conll14st-test-data/noalt/official-2014.combined.m2_id1_corrected.txt',
                      id=1)

In [ ]:
data_id1 = process_m2file(m2_file='/content/drive/My Drive/Grammatical_Error_correction/Data/conll14st-test-data/noalt/official-2014.combined.m2',
                      out_file='/content/drive/My Drive/Grammatical_Error_correction/Data/conll14st-test-data/noalt/official-2014.combined.m2_id11_corrected.txt',
                      id=1)

In [ ]:
data_id1.shape

(1313, 3)

In [ ]:
data_id1.to_csv('/content/drive/MyDrive/Grammatical_Error_correction/Data/csv_data/conll14_combined_id1.csv', index=False)

#Lang8 data set

In [ ]:
!tar -xvzf '/content/drive/My Drive/Grammatical_Error_correction/Data/lang8.bea19.tar.gz' -C '/content/drive/My Drive/Grammatical_Error_correction/Data/lang8'

lang8.train.auto.bea19.m2
lang8_to_m2.py
readme.txt


In [ ]:
data_lang8_id0 = process_m2file(m2_file='/content/drive/MyDrive/Grammatical_Error_correction/Data/lang8/lang8.train.auto.bea19.m2',
                            out_file='/content/drive/MyDrive/Grammatical_Error_correction/Data/lang8/lang8.train.auto.bea19.m2_id0_corrected.txt',
                            id=0)

Preparing dataframe


In [ ]:
data_lang8_id0.head()

,correct,incorrect,total_edits
0,Good luck on your new start !,Good luck on your new start !,0
1,My teacher is going to move to change his job .,My teacher is going to move to change his job .,0
2,He is a so nice guy and taught me English very...,He is a so nice guy and taught me English very...,0
3,And he took in my favorite subjects like soccer .,And he took in my favorite subject like soccer .,1
4,"Actually , he was the one who let me know abou...","Actually , who let me know about Lang - 8 was ...",2


In [ ]:
data_lang8_id0.shape

(1037562, 3)

In [ ]:
data_lang8_id0.to_csv('/content/drive/MyDrive/Grammatical_Error_correction/Data/csv_data/lang8/lang8_anno_id_0.csv', index=False)

Observation: <br>
We have 10 lacks od sentences which are annotated by annotator  with id_0

In [ ]:
data_lang8_id1 = process_m2file(m2_file='/content/drive/MyDrive/Grammatical_Error_correction/Data/lang8/lang8.train.auto.bea19.m2',
                            out_file='/content/drive/MyDrive/Grammatical_Error_correction/Data/lang8/lang8.train.auto.bea19.m2_id1_corrected.txt',
                            id=1)

Preparing dataframe


In [ ]:
data_lang8_id1.head()

,correct,incorrect,total_edits
0,Good luck on your new start !,Good luck on your new start !,0
1,My teacher is going to move to change his job .,My teacher is going to move to change his job .,0
2,He is a so nice guy and taught me English very...,He is a so nice guy and taught me English very...,0
3,And he took in my favorite subject like soccer .,And he took in my favorite subject like soccer .,0
4,"Actually , who let me know about Lang - 8 was ...","Actually , who let me know about Lang - 8 was ...",0


In [ ]:
data_lang8_id1.shape

(1037562, 3)

Observation: <br>
we have 10 lacks of sentences which are annotated by annotator with id_1

In [ ]:
data_lang8_id1.to_csv('/content/drive/MyDrive/Grammatical_Error_correction/Data/csv_data/lang8/lang8_anno_id1.csv', index=False)

In [ ]:
data_lang8_id1 = pd.read_csv('/content/drive/MyDrive/Grammatical_Error_correction/Data/csv_data/lang8/lang8_anno_id1.csv')